# Data Pre-Processing

- This notebook contains all the data cleaning done to acheive our final dataset of `filtered_data`

In [1]:
!pip install statsmodels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

^C


In [2]:
"""
Variable Descriptions

legId: An identifier for the flight.
searchDate: The date (YYYY-MM-DD) on which this entry was taken from Expedia.
flightDate: The date (YYYY-MM-DD) of the flight.
startingAirport: Three-character IATA airport code for the initial location.
destinationAirport: Three-character IATA airport code for the arrival location.
fareBasisCode: The fare basis code.
travelDuration: The travel duration in hours and minutes.
elapsedDays: The number of elapsed days (usually 0).
isBasicEconomy: Boolean for whether the ticket is for basic economy.
isRefundable: Boolean for whether the ticket is refundable.
isNonStop: Boolean for whether the flight is non-stop.
baseFare: The price of the ticket (in USD).
totalFare: The price of the ticket (in USD) including taxes and other fees.
seatsRemaining: Integer for the number of seats remaining.
totalTravelDistance: The total travel distance in miles. This data is sometimes missing.
segmentsDepartureTimeEpochSeconds: String containing the departure time (Unix time) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsDepartureTimeRaw: String containing the departure time (ISO 8601 format: YYYY-MM-DDThh:mm:ss.000±[hh]:00) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsArrivalTimeEpochSeconds: String containing the arrival time (Unix time) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsArrivalTimeRaw: String containing the arrival time (ISO 8601 format: YYYY-MM-DDThh:mm:ss.000±[hh]:00) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsArrivalAirportCode: String containing the IATA airport code for the arrival location for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsDepartureAirportCode: String containing the IATA airport code for the departure location for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsAirlineName: String containing the name of the airline that services each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsAirlineCode: String containing the two-letter airline code that services each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsEquipmentDescription: String containing the type of airplane used for each leg of the trip (e.g. "Airbus A321" or "Boeing 737-800"). The entries for each of the legs are separated by '||'.
segmentsDurationInSeconds: String containing the duration of the flight (in seconds) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsDistance: String containing the distance traveled (in miles) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsCabinCode: String containing the cabin for each leg of the trip (e.g. "coach"). The entries for each of the legs are separated by '||'.
"""
data = pd.read_csv("/work/kaggle_flight_prices/ca_sample_data.csv")
data

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,510f3a1255b9843a8eae48032b191faf,2022-04-17,2022-04-22,OAK,LAX,YH0OASMR,PT7H11M,0,False,False,...,1650678600||1650697140,2022-04-22T18:50:00.000-07:00||2022-04-22T23:5...,SEA||ONT,OAK||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Embraer 175||Airbus A320,7320||8940,672||956,coach||coach
1,5f7a29384cea410317ca308d2e065059,2022-04-17,2022-05-06,SFO,BOS,E0AJZNN1,PT8H29M,1,False,False,...,1651919400||1651929660,2022-05-07T06:30:00.000-04:00||2022-05-07T09:2...,JFK||BOS,SFO||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Airbus A319-321||AIRBUS INDUSTRIE A321 SHARKLETS,20280||4560,2566||185,coach||coach
2,a5e9d9b01627d1e1c54d6b6cbf143945,2022-04-18,2022-04-30,OAK,DTW,QA3OA0MC,PT10H12M,0,False,False,...,1651345740||1651367940||1651376220,2022-04-30T13:09:00.000-06:00||2022-04-30T20:1...,SLC||ORD||DTW,OAK||SLC||ORD,Delta||United||United,DL||UA||UA,Airbus A220-100||Embraer 175 (Enhanced Winglet...,6240||11940||5220,588||1251||240,coach||coach||coach
3,b8714828ff605dfc10a9c511917f1ee8,2022-04-16,2022-04-26,SFO,MIA,R7AZZNN3,PT10H48M,1,False,False,...,1651053360||1651072080,2022-04-27T05:56:00.000-04:00||2022-04-27T11:0...,JFK||MIA,SFO||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Airbus A319-321||Boeing 737 MAX 8,20160||11580,2566||1104,coach||coach
4,80e97c74e379451453a1151b70aaf371,2022-04-17,2022-05-14,LAX,ORD,WAA7OWEN,PT4H4M,1,False,False,...,1652612580,2022-05-15T06:03:00.000-05:00,ORD,LAX,United,UA,Boeing 737-900,14640,1745,coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880262,3bb39f976c56ba93dfab318eadb0bfeb,2022-10-05,2022-11-10,LAX,LGA,KAA4AWEN,PT11H47M,0,False,False,...,1668100920||1668134820,2022-11-10T10:22:00.000-07:00||2022-11-10T21:4...,DEN||LGA,LAX||DEN,United||United,UA||UA,Boeing 737-900||Airbus A319,8520||13020,848||1632,coach||coach
880263,704001c0a3a4cd9b91b7e70cef9d5658,2022-10-05,2022-11-08,SFO,ATL,KA0NA0MQ,PT6H38M,1,False,False,...,1667973180||1667991600,2022-11-08T21:53:00.000-08:00||2022-11-09T06:0...,LAX||ATL,SFO||LAX,Delta||Delta,DL||DL,Airbus A319||,5460||15300,339||1943,coach||coach
880264,60f0ec85cf33eb7efff2aa506d99af30,2022-10-05,2022-11-05,SFO,DFW,VAVNA0BQ,PT7H18M,0,True,False,...,1667645760||1667659380,2022-11-05T05:56:00.000-05:00||2022-11-05T09:4...,MSP||DFW,SFO||MSP,Delta||Delta,DL||DL,Boeing 737-900||,12660||9480,1586||854,coach||coach
880265,839954f5afe360146c028eefe96a6d2a,2022-10-05,2022-11-03,OAK,LGA,NH4OAJMN,PT18H34M,1,False,False,...,1667505900||1667534220||1667564940,2022-11-03T13:05:00.000-07:00||2022-11-03T23:5...,SEA||BOS||LGA,OAK||SEA||BOS,Alaska Airlines||Alaska Airlines||Delta,AS||AS||DL,Boeing 737-900||Boeing 737-800||Embraer 175,7800||19020||5340,672||2489||186,coach||coach||coach


In [3]:
# Only looking at non-stop flights (no layovers)
filtered_data = data[data['segmentsDepartureTimeEpochSeconds'].str.count('\|\|') ==0]

In [4]:
# Extracting the bookingClassCode (indicates economy, business, first class, etc.) from the fareBasisCode
filtered_data['bookingClassCode'] = filtered_data['fareBasisCode'].str[0]

/tmp/ipykernel_526/301374294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['bookingClassCode'] = filtered_data['fareBasisCode'].str[0]


In [5]:
# Casting columns to DateTime object
filtered_data["searchDate"] = pd.to_datetime(filtered_data["searchDate"])
filtered_data['flightDate'] = pd.to_datetime(filtered_data['flightDate'])

# Extracting daysTillFlight from flightDate - searchDate
filtered_data["daysTillFlight"] = (filtered_data["flightDate"] - filtered_data["searchDate"]).dt.days

/tmp/ipykernel_526/3208738070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["searchDate"] = pd.to_datetime(filtered_data["searchDate"])
/tmp/ipykernel_526/3208738070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['flightDate'] = pd.to_datetime(filtered_data['flightDate'])
/tmp/ipykernel_526/3208738070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [6]:
filtered_data = filtered_data.copy()

# Extracting departureDayOfWeek and departureHour from segmentsDepartureTimeRaw
filtered_data["segmentsDepartureTimeRaw"] = pd.to_datetime(filtered_data["segmentsDepartureTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsDepartureTimeRaw"], inplace=True)
filtered_data["segmentsDepartureTimeRaw"] = pd.to_datetime(filtered_data["segmentsDepartureTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsDepartureTimeRaw"], inplace=True)
filtered_data["departureDayOfWeek"] = filtered_data["segmentsDepartureTimeRaw"].dt.dayofweek
filtered_data["departureHour"] = filtered_data["segmentsDepartureTimeRaw"].dt.hour

# Extracting arrivalDayOfWeek and arrivalHour from segmentsArrivalTimeRaw
filtered_data["segmentsArrivalTimeRaw"] = pd.to_datetime(filtered_data["segmentsArrivalTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsArrivalTimeRaw"], inplace=True)
filtered_data["segmentsArrivalTimeRaw"] = pd.to_datetime(filtered_data["segmentsArrivalTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsArrivalTimeRaw"], inplace=True)
filtered_data["arrivalDayOfWeek"] = filtered_data["segmentsArrivalTimeRaw"].dt.dayofweek
filtered_data["arrivalHour"] = filtered_data["segmentsArrivalTimeRaw"].dt.hour

/tmp/ipykernel_526/3623542275.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  filtered_data["segmentsDepartureTimeRaw"] = pd.to_datetime(filtered_data["segmentsDepartureTimeRaw"], errors='coerce')
/tmp/ipykernel_526/3623542275.py:10: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  filtered_data["segmentsArrivalTimeRaw"] = pd.to_datetime(filtered_data["segmentsArrivalTimeRaw"], errors='coerce')


In [7]:
# Casting columns as integers and dropping NaN values
filtered_data['segmentsDurationInSeconds'] = filtered_data['segmentsDurationInSeconds'].astype(int)
filtered_data.dropna(subset=["segmentsDistance"], inplace=True)
filtered_data['segmentsDistance'] = filtered_data['segmentsDistance'].astype(int)

In [8]:
# Drop all unnecessary columns after finishing data cleaning / pre-processing
filtered_data = filtered_data.drop(columns=['legId', 'totalFare', 'fareBasisCode', 'travelDuration', 'segmentsDepartureTimeRaw', 'segmentsArrivalTimeRaw', 'segmentsDepartureTimeEpochSeconds', 'segmentsArrivalTimeEpochSeconds'])

In [9]:
# Final cleaned dataset. Also saved as filtered_data.csv, which is used by some models
filtered_data

,startingAirport,destinationAirport,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,...,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,bookingClassCode,daysTillFlight,departureDayOfWeek,departureHour,arrivalDayOfWeek,arrivalHour
4,LAX,ORD,1,False,False,True,236.28,9,1745.0,ORD,...,Boeing 737-900,14640,1745,coach,W,27,5,23,6,6
70,LAX,ORD,0,False,False,True,305.12,6,1745.0,ORD,...,Boeing 757-300,14880,1745,coach,Q,3,1,10,1,16
109,LAX,ORD,0,False,False,True,322.79,5,1745.0,ORD,...,Boeing 737-800,14700,1745,coach,M,1,1,13,1,19
170,LAX,ORD,0,False,False,True,459.53,7,1745.0,ORD,...,Boeing 737-800,14640,1745,coach,H,8,0,7,0,13
239,LAX,DFW,0,False,False,True,252.09,7,1238.0,DFW,...,Airbus A321,10860,1238,coach,L,5,5,18,5,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879419,LAX,JFK,1,False,False,True,115.35,7,2414.0,JFK,...,Airbus A321,18300,2414,coach,S,31,5,23,6,6
879568,LAX,DFW,0,False,False,True,290.23,7,1193.0,DFW,...,Airbus A321,10620,1193,coach,L,27,1,10,1,15
879716,LAX,ORD,0,False,False,True,245.58,7,1745.0,ORD,...,Boeing 737-800,14940,1745,coach,L,30,4,7,4,13
879987,SFO,ORD,0,True,False,True,171.16,7,1847.0,ORD,...,AIRBUS INDUSTRIE A321 SHARKLETS,15480,1847,coach,N,31,5,10,5,16


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=36e46b6c-b77a-4f8c-a857-c3dc2e354e3d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>